# Understanding Memory in LLMs

In the previous Notebooks, we successfully explored how OpenAI models can enhance the results from Azure AI Search queries. 

However, we have yet to discover how to engage in a conversation with the LLM. With [Bing Chat](http://chat.bing.com/), for example, this is possible, as it can understand and reference the previous responses.

There is a common misconception that LLMs (Large Language Models) have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

In this Notebook, our goal is to illustrate how we can effectively "endow the LLM with memory" by employing prompts and context.

In [1]:
import os
import random
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from typing import List

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import CustomAzureSearchRetriever, get_answer
from common.prompts import DOCSEARCH_PROMPT

from dotenv import load_dotenv
load_dotenv("credentials.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [3]:
QUESTION = "Tell me some use cases for reinforcement learning"
FOLLOW_UP_QUESTION = "What was my prior question?"

In [4]:
COMPLETION_TOKENS = 1000
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [5]:
# We create a very simple prompt template, just the question as is:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}")
])

In [6]:
# Let's see what the GPT model responds
chain = prompt | llm | output_parser
response_to_initial_question = chain.invoke({"input": QUESTION})
display(Markdown(response_to_initial_question))

Reinforcement learning has a wide range of applications across various fields. Some common use cases include:

1. **Game playing**: Reinforcement learning has been successfully used to train agents to play complex games like Chess, Go, and video games. For example, AlphaGo, developed by DeepMind, used reinforcement learning to become the world champion in the game of Go.

2. **Robotics**: Reinforcement learning is used to train robots to perform complex tasks such as grasping objects, navigating environments, and manipulating objects. This enables robots to learn from experience and improve their performance over time.

3. **Autonomous vehicles**: Reinforcement learning is used to train autonomous vehicles to make decisions in real-time while navigating through traffic, following traffic rules, and avoiding collisions. This technology is crucial for the development of self-driving cars.

4. **Recommendation systems**: Reinforcement learning can be used to optimize recommendation systems by learning user preferences and providing personalized recommendations for products, movies, music, and more.

5. **Finance**: Reinforcement learning is used in algorithmic trading to make decisions on buying, selling, and holding financial assets based on market conditions. It can also be used for risk management and portfolio optimization.

6. **Healthcare**: Reinforcement learning can be used to optimize treatment plans for patients by learning from patient data and adjusting treatment strategies to maximize patient outcomes.

7. **Resource management**: Reinforcement learning can be used to optimize resource allocation in various industries such as energy, logistics, and manufacturing. It can help in scheduling tasks, managing inventory, and improving efficiency.

8. **Natural language processing**: Reinforcement learning can be used to train chatbots and virtual assistants to interact with users in a more natural and intelligent way. It can also be used for language translation and text summarization.

These are just a few examples of the many use cases for reinforcement learning. It is a versatile and powerful technique that can be applied to a wide range of problems in different domains.

In [7]:
#Now let's ask a follow up question
printmd(chain.invoke({"input": FOLLOW_UP_QUESTION}))

Your prior question was "What was my prior question?"

As you can see, it doesn't remember what it just responded, sometimes it responds based only on the system prompt, or just randomly. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [8]:
hist_prompt = ChatPromptTemplate.from_template(
"""
    {history}
    Human: {question}
    AI:
"""
)
chain = hist_prompt | llm | output_parser

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response_to_initial_question)

In [10]:
printmd(chain.invoke({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

Your prior question was "Tell me some use cases for reinforcement learning".

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

From Langchain website:
    
A memory system needs to support two basic actions: reading and writing. Recall that every chain defines some core execution logic that expects certain inputs. Some of these inputs come directly from the user, but some of these inputs can come from memory. A chain will interact with its memory system twice in a given run.

    AFTER receiving the initial user inputs but BEFORE executing the core logic, a chain will READ from its memory system and augment the user inputs.
    AFTER executing the core logic but BEFORE returning the answer, a chain will WRITE the inputs and outputs of the current run to memory, so that they can be referred to in future runs.
    
So this process adds delays to the response, but it is a necessary delay :)

![image](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

In [11]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
index3_name = "cogsrch-index-books"
indexes = [index1_name, index2_name, index3_name]

In [12]:
# Initialize our custom retriever 
retriever = CustomAzureSearchRetriever(indexes=indexes, topK=10, reranker_threshold=1)

If you check closely in prompts.py, there is an optional variable in the `DOCSEARCH_PROMPT` called `history`. Now it is the time to use it. It is basically a place holder were we will inject the conversation in the prompt so the LLM is aware of it before it answers.

**Now let's add memory to it:**

In [13]:
store = {} # Our first memory will be a dictionary in memory

# We have to define a custom function that takes a session_id and looks somewhere
# (in this case in a dictionary in memory) for the conversation
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [14]:
# We use our original chain with the retriever but removing the StrOutputParser
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "history": itemgetter("history")
    }
    | DOCSEARCH_PROMPT
    | llm
)

## Then we pass the above chain to another chain that adds memory to it

output_parser = StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
) | output_parser

In [15]:
# This is where we configure the session id
config={"configurable": {"session_id": "abc123"}}

Notice below, that we are adding a `history` variable in the call. This variable will hold the chat historywithin the prompt.

In [16]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Reinforcement learning (RL) has various applications across different domains. Here are some specific use cases for reinforcement learning:

1. **Routing Algorithms**: Reinforcement learning algorithms can be used for routing in networks. These algorithms learn and adapt routing table entries based on feedback from the environment to optimize network traffic flow and minimize delays<sup><a href="https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/9905/9905015v1.pdf" target="_blank">[1]</a></sup>.

2. **Personalized Music Recommendation**: RL can be applied to personalized music recommendation systems. By simulating the interaction process of listeners and continuously updating the recommendation model based on their preferences, RL can provide better song sequences that match listeners' preferences<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/" target="_blank">[2]</a></sup>.

3. **Epidemic Modeling**: RL can be used in microscopic epidemic modeling to consider the consequences of individual decisions on the spread of diseases. By minimizing agents' cost functions and solving for optimal decisions, RL can predict disease spread and identify the need for external interventions<sup><a href="https://arxiv.org/pdf/2004.12959v1.pdf" target="_blank">[3]</a></sup>.

4. **Adaptive Control**: Adaptive control systems can benefit from RL algorithms to improve decision-making processes based on experience. These systems are concerned with improving sequences of decisions in dynamic systems, and RL can enhance these algorithms<sup><a href="https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/9605/9605103v1.pdf" target="_blank">[4]</a></sup>.

5. **Multi-Agent Systems**: RL can be applied in multi-agent systems to enable agents to learn and adapt their behaviors based on interactions with the environment and other agents. This can lead to more efficient and collaborative decision-making processes<sup><a href="https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0306/0306091v1.pdf" target="_blank">[5]</a></sup>.

These examples demonstrate the versatility and effectiveness of reinforcement learning in various real-world applications.

In [17]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was about "Tell me some use cases for reinforcement learning."

In [18]:
# Remembers
printmd(chain_with_history.invoke({"question": "Thank you! Good bye"},config=config))

You're welcome! If you have any more questions in the future, feel free to ask. Goodbye!

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wish to provide recommendations in the future. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory

In [19]:
# Create the function to retrieve the conversation

def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [20]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
) | output_parser

In [21]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

In [22]:
config

{'configurable': {'session_id': 'session996', 'user_id': 'user213'}}

In [23]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Reinforcement learning (RL) has various applications across different domains. Some notable use cases for reinforcement learning include:

1. **Routing Algorithms**: Reinforcement learning algorithms are increasingly used in routing applications. For example, ant-based algorithms and stigmergetic routing frameworks leverage reinforcement learning for learning routing table entries based on network conditions and performance metrics<sup><a href="https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0207/0207073v1.pdf" target="_blank">[1]</a></sup>.

2. **Personalized Recommendation Systems**: Reinforcement learning can be applied to personalized recommendation systems, such as music recommendation algorithms. By simulating the interaction process with users and continuously updating models based on preferences, reinforcement learning can enhance the accuracy of song sequence recommendations<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7206183/" target="_blank">[2]</a></sup>.

3. **Multi-Agent Systems**: Reinforcement learning is used in multi-agent systems for solving complex problems. In routing, for instance, RL algorithms help routers make optimal forwarding decisions based on reinforcement signals from the environment<sup><a href="https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/9905/9905015v1.pdf" target="_blank">[3]</a></sup>.

4. **Game Playing**: RL has been extensively applied in game playing scenarios. For example, in chess, reinforcement learning can be used to learn strategies by analyzing gameplay data and making decisions based on rewards and outcomes<sup><a href="https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0004/0004001v1.pdf" target="_blank">[4]</a></sup>.

5. **Adaptive Control**: Adaptive control systems leverage reinforcement learning algorithms to improve decision-making processes based on experience. These systems are used in dynamic environments where actions and states are vectors, and the system dynamics are smooth, allowing for adaptive learning and control<sup><a href="https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/9605/9605103v1.pdf" target="_blank">[5]</a></sup>.

These examples showcase the versatility and effectiveness of reinforcement learning in solving a wide range of problems across different fields.

In [24]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was about the various use cases of reinforcement learning.

In [25]:
# Remembers
printmd(chain_with_history.invoke(
    {"question": "Can you tell me a one line summary of our conversation?"},
    config=config))

We discussed various use cases of reinforcement learning and then you requested a one-line summary of our conversation.

In [27]:
try:
    printmd(chain_with_history.invoke(
    {"question": "Thank you very much!"},
    config=config))
except Exception as e:
    print(e)

Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, you requested 17216 tokens (15716 in the messages, 1500 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


In [28]:
printmd(chain_with_history.invoke(
    {"question": "I do have one more question, why did you give me a one line summary?"},
    config=config))

I provided a one-line summary in response to your request for a concise overview of our conversation.

In [29]:
printmd(chain_with_history.invoke(
    {"question": "why not 2?"},
    config=config))

I provided a one-line summary as you specifically asked for a one-line summary of our conversation.

#### Let's check our Azure CosmosDB to see the whole conversation


![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, regardless of the input. This doesn't seem efficient, but regardless, we are very close to finish our first RAG-talk to your data Bot.


## <u>Important Note</u>:<br>
As we proceed, while all the code will remain compatible with GPT-3.5 models (1106 or newer), we highly recommend transitioning to GPT-4. Here's why:

**GPT-3.5-Turbo** can be likened to a 7-year-old child. You can provide it with concise instructions, but it struggles sometimes to follow them accurately (not too reliable). Additionally, its limited "memory" (token context) can make sustained conversations challenging. Its response are also simple not deep.

**GPT-4-Turbo** exhibits the capabilities of a 10-12-year-old child. It possesses enhanced reasoning skills, consistently adheres to instructions and its answers are beter. It has extended memory retention (larger context size) for instructions, and it excels at following them. Its responses are deep and thorough.


# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

In the next notebook 6, we are going to build our first RAG bot. In order to do this we will introduce the concept of Agents.